In [1]:
import pandas as pd
import re
import underthesea
from string import punctuation
import warnings
import numpy as np
punctuation = punctuation + '‘’“”'
vietnamese_words = 'aAàÀảẢãÃáÁạẠăĂằẰẳẲẵẴắẮặẶâÂầẦẩẨẫẪấẤậẬbBcCdDđĐeEèÈẻẺẽẼéÉẹẸêÊềỀểỂễỄếẾệỆfFgGhHiIìÌỉỈĩĨíÍịỊjJkKlLmMnNoOòÒỏỎõÕóÓọỌôÔồỒổỔỗỖốỐộỘơƠờỜởỞỡỠớỚợỢpPqQrRsStTuUùÙủỦũŨúÚụỤưƯừỪửỬữỮứỨựỰvVwWxXyYỳỲỷỶỹỸýÝỵỴzZ'
warnings.filterwarnings('ignore')

In [2]:
def read_data(path):
    doc = open(path, encoding='utf-8').read().split('\n')
    text = []
    label = []
    i = 1
    while i < len(doc):
        text.append(doc[i])
        label.append(doc[i + 1])
        i += 4

    df = pd.DataFrame({'review': text, 'sentiment': label})
    return df


In [3]:
def adding_label(data):
    label = data["sentiment"].values

    clean_label = []

    for i in range(len(label)):
        sample = label[i].split(",")
        for j in range(len(sample)):
            sample[j] = re.sub("[{}]", "", sample[j].strip())
        clean_label.append(sample)

    y = []
    for i in range(len(clean_label)):
        row = clean_label[i]

        j = 0
        while j < len(row):
            if(row[j+1] == 'positive'):
                y.append([i + 1 , row[j], 1])
            elif(row[j+1] == 'negative'):
                y.append([i + 1 , row[j], -1])
            else:
                y.append([i + 1 , row[j], 0])
            j += 2
    return y

In [16]:
def data_with_sparse_matrix(df):
    all_classes = ["Text"]
    all_classes += list(set([i[1] for i in adding_label(df)]))
    y = adding_label(df)
    new_dat = pd.DataFrame(columns=all_classes)
    all_classes.remove("Text")
    new_dat.Text = df.review
    new_dat[all_classes] = 0
    sparse_matrix = np.zeros((len(df), len(all_classes)))
    dict_class = {}
    index = 0
    for i in all_classes:
        dict_class[i] = index
        index += 1
    sparse_matrix = list(sparse_matrix.astype(int))
    for i in y:
        sparse_matrix[i[0] - 1][dict_class[i[1]]] = i[2]
    for i in y:
        new_dat[i[1]][i[0] - 1] = i[2]
    final_dat = pd.DataFrame({"Text": new_dat.Text, "Label": sparse_matrix})

    data_with_all_classes = new_dat
    data_with_sparse = final_dat
    for i in range(len(final_dat.Label)):
        s = np.sum(final_dat.Label[i])
        if(s>1):
            final_dat.Label[i] = 1
        elif(s<-1):
            final_dat.Label[i] = -1
        else:
            final_dat.Label[i] = 0
    return data_with_sparse, data_with_all_classes

In [5]:
df = read_data('Dataset_For_Work\\1-VLSP2018-SA-Hotel-train (7-3-2018).txt')

In [6]:
df.head(10)

,review,sentiment
0,Rộng rãi KS mới nhưng rất vắng. Các dịch vụ ch...,"{HOTEL#DESIGN&FEATURES, positive}, {HOTEL#GENE..."
1,"Địa điểm thuận tiện, trong vòng bán kính 1,5km...","{LOCATION#GENERAL, positive}"
2,"Phục vụ, view đẹp, vị trí","{SERVICE#GENERAL, positive}, {HOTEL#GENERAL, p..."
3,"thuận tiện , sạch sẽ , vui vẻ hài lòng","{HOTEL#COMFORT, positive}, {HOTEL#CLEANLINESS,..."
4,Vị trí đẹp; Có quán bar view đẹp; Nhân viên th...,"{LOCATION#GENERAL, positive}, {FACILITIES#GENE..."
5,- Co view huong Ho tay- sach se-nhan vien tan ...,"{HOTEL#GENERAL, positive}, {HOTEL#CLEANLINESS,..."
6,"Phòng ốc sạch, giường thoải mái, nhân viên thâ...","{ROOMS#CLEANLINESS, positive}, {ROOM_AMENITIES..."
7,"gần Hồ Tây, view nhìn ra hồ lãng mạn, đi bộ dọ...","{LOCATION#GENERAL, positive}, {HOTEL#GENERAL, ..."
8,Hình thức không thể quyết định nội dung.H&atil...,"{HOTEL#COMFORT, positive}"
9,Tôi ở đây lần này là lần thứ 4. Khi nhận phòng...,"{ROOMS#CLEANLINESS, negative}, {ROOM_AMENITIES..."


In [7]:
all_classes = list(set([i[1] for i in adding_label(df)]))
y = adding_label(df)

In [8]:
len(all_classes)

33

In [8]:
all_classes[:10]

['FACILITIES#COMFORT',
 'FOOD&DRINKS#PRICES',
 'FACILITIES#DESIGN&FEATURES',
 'ROOM_AMENITIES#GENERAL',
 'HOTEL#COMFORT',
 'SERVICE#GENERAL',
 'FACILITIES#PRICES',
 'ROOM_AMENITIES#COMFORT',
 'ROOM_AMENITIES#MISCELLANEOUS',
 'ROOM_AMENITIES#CLEANLINESS']

In [11]:
y[:10]

[[1, 'HOTEL#DESIGN&FEATURES', 1],
 [1, 'HOTEL#GENERAL', -1],
 [2, 'LOCATION#GENERAL', 1],
 [3, 'SERVICE#GENERAL', 1],
 [3, 'HOTEL#GENERAL', 1],
 [3, 'LOCATION#GENERAL', 1],
 [4, 'HOTEL#COMFORT', 1],
 [4, 'HOTEL#CLEANLINESS', 1],
 [4, 'SERVICE#GENERAL', 1],
 [5, 'LOCATION#GENERAL', 1]]

In [40]:
final_dat,new_dat = data_with_sparse_matrix(df)

In [18]:
final_dat.head()

,Text,Label
0,Rộng rãi KS mới nhưng rất vắng. Các dịch vụ ch...,0
1,"Địa điểm thuận tiện, trong vòng bán kính 1,5km...",0
2,"Phục vụ, view đẹp, vị trí",1
3,"thuận tiện , sạch sẽ , vui vẻ hài lòng",1
4,Vị trí đẹp; Có quán bar view đẹp; Nhân viên th...,1


In [38]:
def clean_text(df,option=0):
    #Trong này đã bao gồm normalization từ vựng lẫn 
    stop_words = open('Dataset_For_Work\\vietnamese-stopwords.txt','r',encoding='utf-8').read().split('\n')
    for i in range(len(df.Text)):
        
        df.Text[i] = underthesea.text_normalize(df.Text[i])

        if(option==1):
            df.Text[i] = re.sub(f'[^{vietnamese_words}{punctuation}]',' ',df.Text[i]) #Việc này để loại bỏ các ký tự biểu tượng cảm xúc trong review Restaurant

        df.Text[i] = underthesea.sent_tokenize(df.Text[i])

    for i in range(len(df.Text)):
        for j in range(len(df.Text[i])):

            df.Text[i][j] = re.sub(f'[{punctuation}]',' ',df.Text[i][j])

            for s_w in stop_words:
                df.Text[i][j] = re.sub(f' {s_w} ',' ',df.Text[i][j])

    for i in range(len(df.Text)):
        new_text = ''
        for j in range(len(df.Text[i])):
            if(j==(len(df.Text[i]) - 1)):
                new_text+=df.Text[i][j]
            else:
                new_text+=(df.Text[i][j] + ' ')
        df.Text[i] = re.sub('\s+',' ',new_text)

    df.to_csv('clean_data.csv')
    return df

In [1]:
def process_data(data):
    sentences = []
    stop_words = open('Dataset_For_Work\\vietnamese-stopwords.txt','r',encoding='utf-8').read().split('\n')
    for item in data:
        processed_item = item.lower()
        for s_t in stop_words:

            processed_item = re.sub(rf" {s_t} ", ' ', processed_item)

        processed_item = re.sub(f'[{punctuation}]',' ',processed_item)
        processed_item = re.sub(f'\s+',' ',processed_item)
        temp = underthesea.sent_tokenize(processed_item)
        sentences += temp

    tokenize_data = [underthesea.word_tokenize(i) for i in sentences]

    return tokenize_data, sentences

In [41]:
clean_dat = clean_text(final_dat)

In [43]:
clean_dat.head()

,Text,Label
0,Rộng rãi KS vắng Các dịch vụ chất,0
1,Địa thuận tiện vòng kính 1 5 km quán ngon,0
2,Phục vụ view đẹp vị trí,1
3,thuận tiện sạch vui vẻ hài lòng,1
4,Vị trí đẹp Có quán bar view đẹp Nhân viên thân...,1


ĐOẠN NÀY LÀ ĐOẠN EMBEDDING

In [76]:
from transformers import AutoModel, AutoTokenizer
import torch

In [74]:
phobert = AutoModel.from_pretrained("vinai/phobert-base-v2")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [87]:
data = []
label = clean_dat.Label.values

In [93]:
text = 'Gia đình 4 3 đêm lưu trú Cửu Long Phú Quốc resort 25 12 28 12 Bởi sát 24 25 hầu sạn resort phòng Cũng may Nhơn chudu24 tư vấn resort Cửu Long'
text = re.sub(r'[^\w\s]', '', text)
    # Loại bỏ các khoảng trắng thừa
text = ' '.join(text.split())

In [94]:
text

'Gia đình 4 3 đêm lưu trú Cửu Long Phú Quốc resort 25 12 28 12 Bởi sát 24 25 hầu sạn resort phòng Cũng may Nhơn chudu24 tư vấn resort Cửu Long'

In [79]:
with torch.no_grad():
    features = phobert(input_ids)

In [85]:
np.array(features.last_hidden_state[0][0])

array([ 3.83621231e-02,  7.07031906e-01, -1.32022694e-01, -4.75826144e-01,
       -2.49211773e-01,  4.77192551e-03, -2.56216198e-01,  2.88822591e-01,
       -2.25086510e-01,  1.96510240e-01,  1.87013801e-02,  2.62436569e-01,
       -6.99767470e-02,  2.05844194e-01, -5.36816239e-01, -1.59152418e-01,
       -3.16560030e-01, -2.22560346e-01, -8.11658874e-02,  5.38349390e-01,
       -1.25653699e-01,  5.63711114e-02, -2.05149770e-01,  2.20803887e-01,
       -6.04585767e-01,  2.24510923e-01, -1.12946056e-01,  3.90720844e-01,
       -3.26886356e-01, -3.81627321e-01, -1.32624730e-01, -2.67986387e-01,
       -7.04586208e-02,  1.74219310e-01,  4.41044867e-01,  5.10359168e-01,
        6.61538914e-03,  2.92966627e-02,  1.19026445e-01,  4.01787281e-01,
        6.66594803e-02,  1.11806333e-01,  1.07561223e-01, -6.95640147e-02,
        3.84218872e-01, -1.67346805e-01,  4.38435137e-01,  2.26754308e-01,
        3.37797999e-01,  2.89732814e-01,  1.71212718e-01,  1.07670896e-01,
       -3.14168334e-01, -